In [25]:
import yfinance as yf
from pathlib import Path
import yaml

from src.data_loader import extract_field_yf, prepare_prices
from src.data_loader import compute_returns, save_parquet

In [26]:
params = yaml.safe_load(Path("../config/params.yaml").read_text())
tickers = params["universe"]["tickers"]
period = params["data"]["period"]
interval = params["data"]["interval"]
field = params["data"]["field"]
tz = params["data"]["tz"]

path_close = params["paths"]["processed_close"]
path_returns = params["paths"]["processed_returns"]
path_corr = params["paths"]["corr"]

In [27]:
raw = yf.download(tickers, period=period, interval=interval, auto_adjust=False)
closes = extract_field_yf(raw, field=field)
prices = prepare_prices(closes, tz=tz)
prices.head()

[*********************100%***********************]  9 of 9 completed


Ticker,BAC,BK,C,GS,JPM,MS,PNC,USB,WFC
Datetime,,,,,,,,,
2024-10-09 15:30:00+02:00,39.910000,73.459999,63.555000,498.095001,211.550003,107.945000,181.740005,44.650002,57.185001
2024-10-09 16:30:00+02:00,40.185001,73.919998,63.933998,503.790009,213.289993,108.599998,183.149994,45.055000,57.410000
2024-10-09 17:30:00+02:00,40.215000,73.910004,63.993599,504.084991,213.329895,108.750000,183.330002,44.990002,57.415001
2024-10-09 18:30:00+02:00,40.279999,74.048599,64.121902,505.010010,213.910004,108.900002,183.634995,44.974998,57.439999
2024-10-09 19:30:00+02:00,40.095001,73.790001,64.074997,504.109985,213.366104,108.849998,182.891998,44.950001,57.290001


In [28]:
returns = compute_returns(prices, log=True)
corr = returns.corr()

save_parquet(prices, f"../{path_close}")
save_parquet(returns, f"../{path_returns}")
save_parquet(corr, f"../{path_corr}")

print("✅ Saved:")
print(f"- close -> {path_close}")
print(f"- returns -> {path_returns}")
print(f"- corr -> {path_corr}")

✅ Saved:
- close -> data/processed/close_1h.parquet
- returns -> data/processed/returns_1h.parquet
- corr -> data/processed/corr_1h.parquet
